In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/convolve-epoch1/sample_submission.csv
/kaggle/input/convolve-epoch1/train.json
/kaggle/input/convolve-epoch1/test.csv
/kaggle/input/epoch1d/final_data_for_training.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
data = pd.read_csv("/kaggle/input/epoch1d/final_data_for_training.csv")
data.head()

,target,month,day,hour,minute,nums,alpha,capital,small
0,normal,0.068621,0.0,0.0,0.016326,0.000479,0.029706,RAS KERNEL INFO,instruction cache parity error corrected
1,normal,0.068621,0.0,0.0,0.016326,0.000479,0.029706,RAS KERNEL INFO,instruction cache parity error corrected
2,normal,0.068621,0.0,0.0,0.016326,0.000479,0.029706,RAS KERNEL INFO,instruction cache parity error corrected
3,normal,0.068621,0.0,0.0,0.016326,0.000479,0.029706,RAS KERNEL INFO,instruction cache parity error corrected
4,normal,0.068621,0.0,0.0,0.016326,0.000479,0.029706,RAS KERNEL INFO,instruction cache parity error corrected


In [4]:
target = data["target"]
data.drop("small", axis=1, inplace=True)

In [5]:
data = pd.concat([data, pd.get_dummies(data["capital"])], axis=1)
data.drop("capital", inplace = True, axis=1)

In [6]:
data.head()

,target,month,day,hour,minute,nums,alpha,NULL DISCOVERY ERROR,NULL DISCOVERY INFO,NULL DISCOVERY SEVERE,NULL DISCOVERY WARNING,NULL HARDWARE SEVERE,NULL HARDWARE WARNING,NULL MONITOR FAILURE,RAS APP FATAL,RAS KERNEL FATAL,RAS KERNEL INFO,RAS LINKCARD INFO
0,normal,0.068621,0.0,0.0,0.016326,0.000479,0.029706,0,0,0,0,0,0,0,0,0,1,0
1,normal,0.068621,0.0,0.0,0.016326,0.000479,0.029706,0,0,0,0,0,0,0,0,0,1,0
2,normal,0.068621,0.0,0.0,0.016326,0.000479,0.029706,0,0,0,0,0,0,0,0,0,1,0
3,normal,0.068621,0.0,0.0,0.016326,0.000479,0.029706,0,0,0,0,0,0,0,0,0,1,0
4,normal,0.068621,0.0,0.0,0.016326,0.000479,0.029706,0,0,0,0,0,0,0,0,0,1,0


In [7]:
data.drop(["target"], axis=1, inplace=True)

In [9]:
data.head()

,month,day,hour,minute,nums,alpha,NULL DISCOVERY ERROR,NULL DISCOVERY INFO,NULL DISCOVERY SEVERE,NULL DISCOVERY WARNING,NULL HARDWARE SEVERE,NULL HARDWARE WARNING,NULL MONITOR FAILURE,RAS APP FATAL,RAS KERNEL FATAL,RAS KERNEL INFO,RAS LINKCARD INFO
0,0.068621,0.0,0.0,0.016326,0.000479,0.029706,0,0,0,0,0,0,0,0,0,1,0
1,0.068621,0.0,0.0,0.016326,0.000479,0.029706,0,0,0,0,0,0,0,0,0,1,0
2,0.068621,0.0,0.0,0.016326,0.000479,0.029706,0,0,0,0,0,0,0,0,0,1,0
3,0.068621,0.0,0.0,0.016326,0.000479,0.029706,0,0,0,0,0,0,0,0,0,1,0
4,0.068621,0.0,0.0,0.016326,0.000479,0.029706,0,0,0,0,0,0,0,0,0,1,0


In [10]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, random_state=42)

In [13]:


from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


In [12]:

nb = MultinomialNB()

nb.fit(X_train, y_train)



MultinomialNB()

In [14]:
res = calculate_results(y_test, nb.predict(X_test))

In [15]:
res

{'accuracy': 99.85976353058939,
 'precision': 0.9986055325156765,
 'recall': 0.9985976353058938,
 'f1': 0.9986011420247594}

In [18]:
from sklearn.metrics import classification_report

print(classification_report(y_test, nb.predict(X_test)))

              precision    recall  f1-score   support

    abnormal       0.96      0.96      0.96      7034
      normal       1.00      1.00      1.00    396570

    accuracy                           1.00    403604
   macro avg       0.98      0.98      0.98    403604
weighted avg       1.00      1.00      1.00    403604



In [20]:
res = calculate_results(y_train, nb.predict(X_train))

print(res)

{'accuracy': 99.85210997930035, 'precision': 0.9985249669660101, 'recall': 0.9985210997930035, 'f1': 0.9985229269891677}
